<a href="https://colab.research.google.com/github/valentindbdg/Improve-Yolo-Perfomance-Data-Centric-Approach/blob/main/Model_1_P1_Train_Yolo_model_2000_HxW608x608_batch64_subdivision16_with_Custom_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training Yolo on Custom Data and Improve Model's Performance Using a Data-centric Approach
## Part 1/4: Train a Yolo model on a custom dataset and output predictions dataset

*Summary*:

*   **Part 1: Training a Yolo Model with a custom dataset**
* Part 2: Improving the dataset
* Part 3: Retrain the model with the cleaned dataset
* Part 4: Convert model for deployement

## 1) Setting up Darknet

Darknet is cloned from this repository [AlexeyAB/darknet](https://github.com/AlexeyAB/darknet.git)

In [ ]:
#%cd /content/

In [ ]:
%%capture
!git clone https://github.com/AlexeyAB/darknet.git

In [ ]:
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!make
!chmod +x ./darknet

## 2) Download required packages

### 2.1 Tensorflow

In [ ]:
'''
%tensorflow_version 1.x
import tensorflow as tf
tf.__version__
'''

###2.2 Fityone

In [ ]:
!pip uninstall opencv_python_headless

In [ ]:
!pip install opencv-python-headless==4.5.4.60

In [ ]:
!pip install fiftyone

## 3) Download the data
The coco-2017 [*dataset*](https://cocodataset.org/#download) was downloaded from the official website using the package Fiftyone in order to download only the data from the class "person". The image file and the annotation file were then added to the drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

The data is downloaded

In [ ]:
import fiftyone as fo

In [ ]:
import random

import fiftyone.zoo as foz
from fiftyone import ViewField as F


classes = ["person"]

dataset = foz.load_zoo_dataset(
    "coco-2017",
    split="validation",
    classes=classes,
    only_matching=True,
).filter_labels("ground_truth", F("iscrowd") == 0)

print(dataset.count_values("ground_truth.detections.label"))



##4) Visualize the data
The dataset is visualized using Fiftyone

In [ ]:
# View summary info about the dataset
print(dataset)

In [ ]:
# Print the first few samples in the dataset
print(dataset.head())

In [ ]:
session = fo.launch_app(dataset)

##5) Preprocess the dataset
A different format is required for Yolo, and a conversion from COCO format to Yolo format is performed. The file is converted to .csv file so pandas can be used to transform the data. The goal is to convert the COCO format into a YOLO format. Values in the annotation Yolo file must be normalized with the width and height of each image concerned.
For each image, a *txt* file with the same name is required. Each *txt* contains one row for each bounding box as presented below:
`<object-class> <x_center> <y_center> <width> <height>`,

with:
* `<object-class>`: ID of the object category [ 0 to (num_of_classes - 1) ] ;
* `<x_center>` and `<y_center>`: the x and y coordinates of the center of the bounding box ;
* `<width>` and `<height>`: the width and height of the bounding box.

Coordinates are normalized with the width and height of the image.

The package Fiftyone is used for this task as it is possible to conveniently convert one format into another

In [ ]:
# convert and export to disk the labels of interest
dataset.export(
    export_dir="/content/yolodataset",
    dataset_type=fo.types.YOLOv4Dataset,
    split="validation",
    classes=classes,
    label_field="ground_truth",
)

The ground_truth dataset is also exported and saved to the drive:

In [ ]:
# convert and export to disk the labels of interest
dataset.export(
    export_dir="/content/drive/MyDrive/yolodataset",
    dataset_type=fo.types.YOLOv4Dataset,
    split="validation",
    classes=classes,
    label_field="ground_truth",
)

In [ ]:
# Now load ground truth labels into a new dataset
dataset = fo.Dataset.from_dir(
    dataset_dir="/content/drive/MyDrive/yolodataset",
    dataset_type=fo.types.YOLOv4Dataset,
    label_field="ground_truth",
)

In [ ]:
# View summary info about the dataset
print(dataset)

In [ ]:
# Print the first few samples in the dataset
print(dataset.head())

In [ ]:
session = fo.launch_app(dataset)

## 6) Prepare the data for training
* The pre-trained weights are downloaded. 
* *obj.names*: the text labels of our objects,
* *obj.data*: the paths to files that define the split + some additional info,
* and *cfg*: the file which contains the configuration for our model.

### 6.1 Train/validation/test split

A train/validation/test split is performed. Three .txt files are created with paths to the images: 
* *train.txt*: paths to the images on which the model is trained ; 
* *val.txt*: paths to the images used for evaluation of the fit during training ;
* *test.txt*: paths to the images used for evaluation of the final model (not seen during training).

A 80% - 20% split for traning and test set is performed. The training set is split again into a validation set and actual train set. Since the whole data is actually traied on the val2017 datasplit and only has more than 2000 images, to allocate a few more images for training, the data is split as described here: 
* train 80% ;
* val 10% ; 
* test 10%.

From txt to csv

Copy yolo ground truth dataset to local directory /content:

In [ ]:
!cp -av '/content/drive/MyDrive/yolodataset' '/content/yolodataset'

In [ ]:
import pandas as pd
df= pd.read_csv ("/content/yolodataset/images.txt")

In [ ]:
df['Frame'] = df['data/000000000139.jpg']

In [ ]:
df['Frame']=df['Frame'].str[5:]

Shuffle the data:

In [ ]:
import random
ids = df["Frame"].unique().tolist()
random.shuffle(ids)

Define the ratios:

In [ ]:
total = len(ids)
limit_train = int(total * 0.8)
limit_val = int(total * 0.9)

Create the .txt files:

In [ ]:
ds_path = '/content/val2017'
def write_list(array, fname):
  textfile = open(fname, "w")
  for element in array:
    textfile.write(f"{ds_path}/{element}\n")
  textfile.close()

In [ ]:
ids_train = ids[:limit_train]
ids_val = ids[limit_train:limit_val]
ids_test = ids[limit_val:]

In [ ]:
%cd /content/
!mkdir data

In [ ]:
!ls

In [ ]:
write_list(ids_train, "/content/data/train.txt")
write_list(ids_val, "/content/data/val.txt")
write_list(ids_test, "/content/data/test.txt")

*train.txt*, *val.txt*, and *test.txt* define the train/val/test split where each line provides the relative path to the image in this split.

In [ ]:
%cd /content/

In [ ]:
import shutil
shutil.copytree('/content/yolodataset/data', '/content/val2017')

### 6.2 Downloading the weights

Download weights for YoloV3-tiny and for YoloV4-tiny.

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.conv.29
!wget https://github.com/GotG/yolotinyv3_medmask_demo/raw/master/yolov3-tiny.conv.15


### 6.3 obj.names

We create the `obj.names`. Each line must be its own text label. We use them straight from the *filter_categories* object, but you can also change the code to provide them in an array, like seen in the comment below.

In [ ]:
labels_path = '/content/obj.names'

# make a list of your labels
# labels = ['person']
filter_categories = ['person']
labels = filter_categories

with open(labels_path, 'w') as f:
    f.write('\n'.join(labels))

#check that the labels file is correct
!cat $labels_path

### 6.4 obj.data
We set up `obj.data`, which needs to include the number of classes, paths to train, validation, and test *txts*, path to the *obj.names*, and the name of the folder were weights will be saved.

**the printed lines must match the data**

In [ ]:
import re
objdata = '/content/obj.data'

#the number of classes is equal to the number of labels
num_classes = len(labels)   

with open(objdata, 'w') as f:
  f.write(f"classes = {num_classes}\n")
  f.write(f"train = /content/data/train.txt\n")
  f.write(f"valid = /content/data/val.txt\n")
  f.write(f"names = /content/obj.names\n")
  f.write(f"backup = backup/")

!cat $objdata

Another file for evaluation is created. As currently there is no support for test set straight from the obj.data, a copy is created and "valid" is set to the test set.

In [ ]:
import re
objdata = '/content/obj_test.data'

#the number of classes is equal to the number of labels
num_classes = len(labels)   

with open(objdata, 'w') as f:
  f.write(f"classes = {num_classes}\n")
  f.write(f"train = /content/data/train.txt\n")
  f.write(f"valid = /content/data/test.txt\n")
  f.write(f"names = /content/obj.names\n")
  f.write(f"backup = backup/")

!cat $objdata

### 6.5 .cfg file

The config file .cfg is copied from the repository and edited to match with the dataset. The Yolo version is chosen in this section.

[yolov4-tiny-custom.cfg](https://github.com/AlexeyAB/darknet/blob/master/cfg/yolov4-tiny-custom.cfg) is used for yolov4 version since it contains correctly set up masks in the `[yolo]` layers.

In [ ]:
!cp /content/darknet/cfg/yolov3-tiny.cfg /content/yolov3-tiny.cfg
!cp /content/darknet/cfg/yolov4-tiny-custom.cfg /content/yolov4-tiny.cfg

####6.5.1 Choosing Yolo version

Choose either Yolo v3 or v4

In [ ]:
yolo_version = 3
cfg_file = f'/content/yolov{yolo_version}-tiny.cfg'
cfg_file

In [ ]:
if yolo_version == 4:
  weights_file = '/content/yolov4-tiny.conv.29'
else:
  weights_file = '/content/yolov3-tiny.conv.15'

In [ ]:
weights_file

#### 6.5.2 Setting up the parameters

Square input sizes are chosen but a custom height and width can be used, as long as both height and width are divisible by 32. 
Preferably, the input size must be similar to the original image size to keep aspect ratio, but use a smaller input for faster inference.

Other parameters in the .cfg files are set according to [this tutorial](https://github.com/AlexeyAB/darknet#how-to-train-to-detect-your-custom-objects). 

In [ ]:
# must be divisible by 32
yolo_height = 608 #608
yolo_width = 608 #608

# set the number of max_batches - min 2000 per class:
max_batch = 2000 #8000
# calculate the 2 steps values:
step1 = 0.8 * max_batch
step2 = 0.9 * max_batch

# we also need to adjust the number of classes and a parameter called filter size 
# that are both is inside the model structure

num_classes = len(labels)
num_filters = (num_classes + 5) * 3

batch = 64
# If out of memory error, increase subdivision (8, 16, 32, or 64) 
subdivisions = 16

In [ ]:
with open(cfg_file) as f:
    s = f.read()
# (re.sub('[a-z]*@', 'ABC@', s))
s = re.sub('max_batches = \d*','max_batches = '+str(max_batch),s)
s = re.sub('steps=\d*,\d*','steps='+"{:.0f}".format(step1)+','+"{:.0f}".format(step2),s)
s = re.sub('classes=\d*','classes='+str(num_classes),s)
s = re.sub('pad=1\nfilters=\d*','pad=1\nfilters='+"{:.0f}".format(num_filters),s)
s = re.sub('batch=\d*', 'batch='+str(batch), s)
s = re.sub('subdivisions=\d*', 'subdivisions='+str(subdivisions), s)
s = re.sub('height=\d*', 'height='+str(yolo_height), s)
s = re.sub('width=\d*', 'width='+str(yolo_width), s)

# pad=1\nfilters=\d\d
# s = re.sub('CUDNN=0','CUDNN=1',s)
# s = re.sub('OPENCV=0','OPENCV=1',s)

with open(cfg_file, 'w') as f:
  # s = re.sub('GPU=0','GPU=1',s)
  f.write(s)



We inspect the config file and check whether the parameters were set correctly.

In [ ]:
!head -n 24 $cfg_file

We also inspect the last few lines to see if the number of filters in [convolutional] before [yolo] is correct.

In [ ]:
!tail -n 64  $cfg_file

#### 6.5.3 Anchors

Anchors can be recalculated to improve detection. The below code only contains information on how to calculate them, and the config file must be edited accordingly. 
Tutorials available [*here*](https://github.com/AlexeyAB/darknet#how-to-train-to-detect-your-custom-objects) and using [*this issue*](https://github.com/AlexeyAB/darknet/issues/7856).

In [ ]:
#%cd /content/darknet/

In [ ]:
#!./darknet detector calc_anchors /content/obj.data -num_of_clusters 6 -width 320 -height 320

## 7) Export to disk the yolo label files
Now that the ground truth labels of all other classes except "person" has been removed and converted to a YOLO format, the annotation file is saved to the drive to be used later in the second notebook of this project: (Part 2)



####7.1 Create a new folders based on the expected format of Fiftyone
In Fiftyone, datasets of this type are read in the following format:
```
<dataset_dir>/
    obj.names
    images.txt
    data/
        <uuid1>.<ext>
        <uuid1>.txt
        <uuid2>.<ext>
        <uuid2>.txt
        ...
```

where obj.names contains the object class labels:
```
<label-0>
<label-1>
'''
```

and images.txt contains the list of images in data/:
```
data/<uuid1>.<ext>
data/<uuid2>.<ext>
...
```

The image paths in images.txt can be specified as either relative (to the location of file) or as absolute paths. Alternatively, this file can be omitted, in which case the data/ directory is listed to determine the available images.

The TXT files in data/ are space-delimited files where each row corresponds to an object in the image of the same name, in the following format:

<target> <x-center> <y-center> <width> <height>
where <target> is the zero-based integer index of the object class label from obj.names and the bounding box coordinates are expressed as relative coordinates in [0, 1] x [0, 1].

Unlabeled images have no corresponding TXT file in data/.

###7.2 Save the files to the new folders



In [ ]:
'''
import os
FOLDER_PATH = 'val2017'
ROOT_PATH = '/content/'
print(len(os.listdir(os.path.join(ROOT_PATH, FOLDER_PATH))))
'''

## 8) Training

The `-map` flag is used to get the best weights based on the highest mAP on the validation set in order to avoid overfitting.
The best weights are saved in /content/darknet/backup/ directory with suffix *best*.

In [ ]:
%cd /content/darknet/

Training:

In [ ]:
!./darknet detector train /content/obj.data $cfg_file $weights_file -dont_show -ext_output -map

## 9) Testing and evaluation

The best weights are used to evaluate the model on the test data set

In [ ]:
weights_best = "/content/darknet/backup/yolov4-tiny_best.weights"
if yolo_version == 3:
  weights_best = "/content/darknet/backup/yolov3-tiny_best.weights"

In [ ]:
weights_best

Copy the best weights to the drive to save it

In [ ]:
!cp '/content/darknet/backup/yolov3-tiny_best.weights' '/content/drive/MyDrive/'

### 9.1 Testing on a random image

Test and visualization of the model on a selected image:

In [ ]:
!./darknet detector test /content/obj.data  $cfg_file  $weights_best /content/val2017/000000371552.jpg -ext_output

The prediction is saved to *predictions.jpg*.

In [ ]:
from google.colab.patches import cv2_imshow
import cv2
!./darknet detector calc_anchors /content/obj.data -num_of_clusters 6 -width 608 -height 608

img = cv2.imread("predictions.jpg")
cv2_imshow(img)

### 9.2 Evaluation on a test data set

The second `obj_test.data` file is used, where `valid` was set to `test.txt`. `map` is used with the .cfg file and the best weights to run the evaluation on that test set.

In [ ]:
!./darknet detector map /content/obj_test.data $cfg_file $weights_best -points 0

The metrics used are detailed in the last few lines:
* Precision ;
* Recall ;
* F1 score ;
* TP ;
* FP ;
* FN ; 
* Average IoU.

###9.3 Evaluate on a test set and output the prediction dataset

In [ ]:
!./darknet detector test /content/obj_test.data $cfg_file $weights_best -dont_show -ext_output </content/data/test.txt> result2.txt

We create a prediction using "valid" to get a file with the output results that can be parsed more easily than if using "test":

In [ ]:
!./darknet detector valid /content/obj_test.data $cfg_file $weights_best -dont_show -ext_output </content/data/test.txt> result3.txt

## 10) Saving model weights

The first YOLO model was succesfully trained. The weights are downloaded and saved so the model does not have to be trained again.

In [ ]:
from google.colab import files
files.download(weights_best)

in the file data/train.txt you should have paths to images
in the file result.txt will be results of detections

## 11) Improvement
Improvements can be performed as suggested in [AlexeyAB/darknet](https://github.com/AlexeyAB/darknet#how-to-improve-object-detection) repository. 
In Part 2, improvements are performed by improving the dataset.

## Save model folders and predictions to google drive

In [ ]:
%cd /content/drive/MyDrive/

In [ ]:
!mkdir yolov3

In [ ]:
!cp '/content/yolov3-tiny.cfg' '/content/drive/MyDrive/yolov3'

In [ ]:
!cp '/content/obj.names' '/content/drive/MyDrive/yolov3'

In [ ]:
!cp '/content/darknet/result2.txt' '/content/drive/MyDrive/yolov3' 

In [ ]:
!cp -av '/content/data' '/content/drive/MyDrive/yolov3' 

In [ ]:
%cd /content/drive/MyDrive/

In [ ]:
!rm -rf yolodataset

In [ ]:
!cp -av '/content/yolodataset' '/content/drive/MyDrive/yolodataset' 